# 🔥 TITAN Radar - RFSoC 4x2 PYNQ Demonstration

**QEDMMA Radar-on-Chip Implementation**

Author: Dr. Mladen Mešter  
Platform: AMD RFSoC 4x2 (Zynq UltraScale+ ZU48DR)

---

## Hardware Specifications

| Parameter | Value |
|-----------|-------|
| ADC | 4× 5 GSPS, 14-bit, DC-6 GHz |
| DAC | 2× 9.85 GSPS, 14-bit |
| Logic Cells | 930,000 |
| DSP Slices | 4,272 |
| Memory | 8 GB DDR4 |
| High-Speed I/O | 100 GbE QSFP28 |

## 1. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import time
import sys

# Add driver path
sys.path.insert(0, '/home/xilinx/titan_radar/drivers')

from titan_radar import TitanRadarOverlay, TitanConfig, WaveformType, CFARMode

print("✅ Libraries imported")

## 2. Configure TITAN Radar

In [ ]:
# Create configuration
config = TitanConfig(
    center_freq_mhz=155.0,      # VHF for anti-stealth
    sample_rate_msps=500.0,     # 500 MSPS after decimation
    chip_rate_mhz=10.0,         # 10 Mchip/s = 15m resolution
    num_range_bins=16384,       # Massive range coverage
    num_doppler_bins=1024,      # Good velocity resolution
    cpi_pulses=64,              # 64 pulses per CPI
    cfar_mode=CFARMode.CA,      # Cell Averaging CFAR
    cfar_threshold_factor=5.0,  # 5× noise threshold
)

config.print_config()

## 3. Initialize TITAN Overlay

In [ ]:
# Create radar overlay
radar = TitanRadarOverlay(
    bitstream='titan_radar.bit',
    config=config
)

# Configure system
radar.configure()

print("\n✅ TITAN Radar initialized")

## 4. Start Radar Operation

In [ ]:
radar.start()
print("📡 Radar transmitting...")

## 5. Process and Display Range-Doppler Map

In [ ]:
# Process one CPI
result = radar.process_cpi(simulate_targets=True)

# Extract data
rd_map = result['range_doppler']
detections = result['detections']

print(f"Detections: {len(detections)}")
for d in detections:
    print(f"  Range: {d.range_m/1000:.1f} km, Velocity: {d.velocity_mps:.0f} m/s, SNR: {d.snr_db:.1f} dB")

In [ ]:
# Plot Range-Doppler Map
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Range-Doppler map
ax1 = axes[0]
range_axis = np.arange(rd_map.shape[1]) * config.range_resolution_m / 1000  # km
doppler_axis = np.linspace(-config.num_doppler_bins//2, config.num_doppler_bins//2, config.num_doppler_bins)

im = ax1.imshow(20*np.log10(rd_map + 1e-10), 
                aspect='auto', 
                extent=[0, range_axis[-1], doppler_axis[0], doppler_axis[-1]],
                cmap='viridis',
                vmin=-30, vmax=30)

# Mark detections
for d in detections:
    ax1.plot(d.range_m/1000, d.doppler_bin - config.num_doppler_bins//2, 
             'ro', markersize=10, markerfacecolor='none', linewidth=2)

ax1.set_xlabel('Range (km)')
ax1.set_ylabel('Doppler Bin')
ax1.set_title('Range-Doppler Map')
ax1.set_xlim([0, 300])  # Show first 300 km
plt.colorbar(im, ax=ax1, label='Power (dB)')

# Range profile (zero Doppler)
ax2 = axes[1]
zero_doppler = rd_map[config.num_doppler_bins//2, :]
ax2.plot(range_axis, 20*np.log10(zero_doppler + 1e-10))
ax2.set_xlabel('Range (km)')
ax2.set_ylabel('Power (dB)')
ax2.set_title('Range Profile (Zero Doppler Cut)')
ax2.set_xlim([0, 300])
ax2.grid(True, alpha=0.3)

# Mark detections
for d in detections:
    ax2.axvline(d.range_m/1000, color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 6. Beamforming Demonstration

In [ ]:
# Compute beam pattern
angles = np.linspace(-90, 90, 361)
pattern = np.zeros(len(angles))

for i, angle in enumerate(angles):
    sv = radar.beamformer.steering_vector(angle)
    pattern[i] = np.abs(radar.beamformer.weights.conj() @ sv)**2

pattern_db = 10 * np.log10(pattern / np.max(pattern) + 1e-10)

plt.figure(figsize=(10, 5))
plt.plot(angles, pattern_db)
plt.xlabel('Angle (degrees)')
plt.ylabel('Gain (dB)')
plt.title('4-Channel Array Beam Pattern @ 155 MHz')
plt.xlim([-90, 90])
plt.ylim([-30, 5])
plt.grid(True, alpha=0.3)
plt.axhline(-3, color='r', linestyle='--', label='-3 dB')
plt.legend()
plt.show()

print(f"Array gain: +{10*np.log10(4):.1f} dB (4 elements)")

In [ ]:
# Steer beam to different angles
steer_angles = [0, 15, 30, -20]

plt.figure(figsize=(10, 5))

for steer in steer_angles:
    radar.set_steering(steer)
    
    pattern = np.zeros(len(angles))
    for i, angle in enumerate(angles):
        sv = radar.beamformer.steering_vector(angle)
        pattern[i] = np.abs(radar.beamformer.weights.conj() @ sv)**2
    
    pattern_db = 10 * np.log10(pattern / np.max(pattern) + 1e-10)
    plt.plot(angles, pattern_db, label=f'Steer: {steer}°')

plt.xlabel('Angle (degrees)')
plt.ylabel('Gain (dB)')
plt.title('Beam Steering Demonstration')
plt.xlim([-90, 90])
plt.ylim([-30, 5])
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## 7. Real-Time Processing Loop

In [ ]:
# Process multiple CPIs and measure performance
num_cpis = 10
times = []
all_detections = []

radar.set_steering(0)  # Reset to broadside

print("Processing CPIs...")
for i in range(num_cpis):
    t_start = time.time()
    result = radar.process_cpi(simulate_targets=True)
    t_end = time.time()
    
    times.append(t_end - t_start)
    all_detections.extend(result['detections'])
    
    print(f"CPI {i+1}: {len(result['detections'])} detections, "
          f"{(t_end-t_start)*1000:.1f} ms")

print(f"\n📊 Performance:")
print(f"   Average: {np.mean(times)*1000:.1f} ms per CPI")
print(f"   Max rate: {1/np.mean(times):.1f} Hz")
print(f"   Total detections: {len(all_detections)}")

## 8. Detection Statistics

In [ ]:
# Plot detection statistics
if all_detections:
    ranges = [d.range_m/1000 for d in all_detections]
    velocities = [d.velocity_mps for d in all_detections]
    snrs = [d.snr_db for d in all_detections]
    
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    
    axes[0].hist(ranges, bins=20, edgecolor='black')
    axes[0].set_xlabel('Range (km)')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Range Distribution')
    
    axes[1].hist(velocities, bins=20, edgecolor='black')
    axes[1].set_xlabel('Velocity (m/s)')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Velocity Distribution')
    
    axes[2].hist(snrs, bins=20, edgecolor='black')
    axes[2].set_xlabel('SNR (dB)')
    axes[2].set_ylabel('Count')
    axes[2].set_title('SNR Distribution')
    
    plt.tight_layout()
    plt.show()

## 9. Cleanup

In [ ]:
radar.stop()
radar.close()
print("✅ Radar shutdown complete")

---

## 📊 Summary

### TITAN Radar Capabilities on RFSoC 4x2:

| Feature | Specification |
|---------|---------------|
| **RF Bandwidth** | DC - 6 GHz (direct sampling) |
| **ADC Rate** | 5 GSPS × 4 channels |
| **DAC Rate** | 9.85 GSPS × 2 channels |
| **Range Bins** | 16,384 |
| **Range Resolution** | 15 m |
| **Max Range** | 2,457 km theoretical |
| **Doppler Bins** | 1,024 |
| **Beamforming** | 4-channel, ±60° steering |
| **Max Tracks** | 256 |
| **Processing** | Real-time on FPGA |
| **Data Offload** | 100 GbE QSFP28 |

### Cost:
- **RFSoC 4x2 Board:** $2,149 (academic)
- **Complete System:** ~€2,800

🔥 **This is a complete RADAR ON A CHIP!**